### Author: Jose Miguel Bautista
### Updated: 05/31/2024

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import time
from functools import cache

# Preface

We’ll spend a few extra sessions on Dynamic Programming because:
1. It is much more about methodology and design than adapting "standard" algorithms.  
1. It is both powerful and flexible, so there are a lot of places to apply it.  
1. Historically, there has been a steep initial learning curve. 
1. It shows up in a LOT of job interviews.

Regarding the last point, I will clarify that it is entirely possible you will take a job with programming and *never* use dynamic programming.  
You *might* if you go to a big tech company, do operations research, or something similar.  
But it shows up in technical interviews, particularly for top-level jobs, because it's a very good litmus test.  
If you understand dynamic programming, you almost certainly know all of data structures and algorithms and (more importantly) how to apply them to a job.  

# Overview of Dynamic Programming

Recall that greedy searches focus on making the best local choice at each decision point.  
Such algorithms (like Dijkstra's algorithm) only work if the solution space is special.  
We said that it is like having no local minima to get trapped in.  
More accurately, it depends on a problem having 2 properties. 
1. *Greedy-choice property:* Choosing the best (local) option at each step will lead to the best overall solution.
1. *Optimal substructure:* The optimal solution to the main problem is made of optimal solutions to smaller sub-problems.  

Usually, when a problem exhibits greedy-choice, it exhibits optimal substructure (though it doesn't have to e.g. finding the maximum of a set).  
But conversely, problems with optimal substructure do not always exhibit greedy choice.  
So barring formal proof for any given problem, greedy algorithms are unlikely to be optimal.  

Exhaustive searches just look at every possible solution with brute force (and occasional tricks like pruning).  
These are guaranteed to work at finding the optimal solution, if they run.  
The problem with these is that depending on the problem, they may be slow or implausible to run in reasonable time.  

**Dynamic programming** is somewhere in between the two.  
It is fundamentally an exhaustive search, so it is going to give the best result (if it runs in finite time).  
But it is applied to problems that have optimal substructure, i.e. without greedy-choice.  
There is one other requirement for it which is that the problem has *overlapping sub-problems*.  
In short, when a problem is broken up into sub-problems, the answers to the subproblems are reused several times.  
For problems with optimal substructure, but non-overlapping subproblems, you would reach for divide-and-conquer.  

We have already seen a few dynamic programming algorithms in the portions of graphs, so I will give the high-level idea of generic dynamic programming.  
1. Express the problem as a recursive one.
1. Identify the base cases, and determine the order in which to solve the higher subproblems.
1. Solve the subproblems in order, storing some intermediate results if they are used in higher subproblems. 






# Example: Fibonacci Numbers

The *Fibonacci sequence* is a sequence of numbers defined by the recursive relationship $$F(n) = F(n-1) + F(n-2)$$ where the sequence starts at $F(1) = 1$ and either $F(0) = 0$ or $F(0) = 1$ depending on who you ask.  
For the work below, I will pick the convention $F(0) = 1$. 

## Naive Recursion 

This is simple enough to code up as a doubly-recursive function, shown in the demo code below.  
It clearly works, though it is incredibly slow and inefficient.  

<img src="img/19-20_Fn.png" style="width: 50em" />  

For example, calculating both $F(6)$ and $F(5)$ will explicitly require $F(4)$.  
But neither function call will know if one of them has already found $F(4)$, as the function calls (on a stack) will be on different branches of the DAG.  
Propagated to the base cases, the naive version will calculate $F(1)$, and $F(0)$, over and over again.  
These base cases (leaf nodes) have value 1, and the ratio of consecutive numbers in the sequence is asymptotically the golden ratio $\phi$ $$\lim_{n\rightarrow \infty} \dfrac{F(n+1)}{F(n)} = \phi \approx 1.6$$  
Thus the time complexity (read: number of leaf nodes) is roughly $O(1.6^{n})$.  


As reflected in the demo code, the naive recursive approach is fine for most low values.  
But due to the exponential scaling behavior, it starts to rapidly struggle around $n\approx 30$.  
You can also verify for yourself that the ratio of subsequent times is $\approx 1.6$, at least at high $n$.  
Even if you're willing to wait a while, remember there is some max stack limit for Python, so this will definitely fail at high enough $n$. 

In [2]:
def F_rec(n):
    if (n == 0)|(n == 1):
        return 1
    else:
        return F_rec(n-1) + F_rec(n-2) 
    
print('   n | time[sec] | F(n)')
for i in range(0, 36): # default 36, go higher if more patient
    start = time.time()
    temp = F_rec(i)
    end = time.time()
    print(str(i).rjust(4), '|', str(np.round(end - start, 6)).rjust(9), '|', str(temp))

   n | time[sec] | F(n)
   0 |     1e-06 | 1
   1 |       0.0 | 1
   2 |       0.0 | 2
   3 |       0.0 | 3
   4 |     1e-06 | 5
   5 |     2e-06 | 8
   6 |     3e-06 | 13
   7 |     4e-06 | 21
   8 |     7e-06 | 34
   9 |   1.1e-05 | 55
  10 |   1.9e-05 | 89
  11 |     3e-05 | 144
  12 |   4.8e-05 | 233
  13 |   7.1e-05 | 377
  14 |  0.000113 | 610
  15 |  0.000185 | 987
  16 |  0.000297 | 1597
  17 |   0.00048 | 2584
  18 |  0.000775 | 4181
  19 |  0.001222 | 6765
  20 |  0.002026 | 10946
  21 |  0.003043 | 17711
  22 |  0.004643 | 28657
  23 |  0.006975 | 46368
  24 |  0.010349 | 75025
  25 |  0.015736 | 121393
  26 |  0.025157 | 196418
  27 |  0.040761 | 317811
  28 |  0.065696 | 514229
  29 |  0.107241 | 832040
  30 |  0.179681 | 1346269
  31 |  0.288904 | 2178309
  32 |  0.461915 | 3524578
  33 |  0.753879 | 5702887
  34 |  1.210485 | 9227465
  35 |  1.967838 | 14930352


## Caching and Top-down Dynamic Programming 

As mentioned, this naive code is slow because in calculating $F(n)$, we need to use $F(n'< n)$ more frequently the lower $n'$ is.  
But the different function calls don't talk to each other.   
So even if one has already calculated it, the other function calls cannot access the same calculation.   

Rather than recalculating every time, it makes more sense to store or *cache* the intermediate calculations in a common area.  
Then each call can just check if a value has already been calculated before and retrieve as needed.  
We are effectively trading some space (allocated to the cache array) for speed (removing redundant calculations).  
This is similar to the cache memory storage on most computers, hence the name.  
In other sources (like Cormen), they refer to this more specifically as *memoization* and the storage space as the *memo* array.  

Below I have some demo code that mostly follows Skiena's format for basic caching.  


In [3]:
def F_cache_driver(n):
    if (n == 0)|(n == 1):
        return 1
    cache = [-1]*(n+1)
    cache[0] = 1
    cache[1] = 1
    def F_cache(n): 
        if (cache[n] == -1):
            cache[n] = F_cache(n-1) + F_cache(n-2) 
        return cache[n]

    return F_cache(n)

print('   n | time[sec] | F(n)')
for i in range(0, 36): # default 36, go higher if more patient
    start = time.time()
    temp = F_cache_driver(i)
    end = time.time()
    print(str(i).rjust(4), '|', str(np.round(end - start, 7)).rjust(9), '|', str(temp))

   n | time[sec] | F(n)
   0 |   1.2e-06 | 1
   1 |       0.0 | 1
   2 |   1.7e-06 | 2
   3 |     1e-06 | 3
   4 |   1.2e-06 | 5
   5 |     1e-06 | 8
   6 |     1e-06 | 13
   7 |     1e-06 | 21
   8 |   1.9e-06 | 34
   9 |     1e-06 | 55
  10 |   1.9e-06 | 89
  11 |   2.1e-06 | 144
  12 |   2.6e-06 | 233
  13 |   1.9e-06 | 377
  14 |   1.9e-06 | 610
  15 |   3.1e-06 | 987
  16 |   2.9e-06 | 1597
  17 |   3.1e-06 | 2584
  18 |   3.1e-06 | 4181
  19 |   3.1e-06 | 6765
  20 |   6.2e-06 | 10946
  21 |   4.1e-06 | 17711
  22 |   4.8e-06 | 28657
  23 |   4.3e-06 | 46368
  24 |   5.2e-06 | 75025
  25 |   4.8e-06 | 121393
  26 |   3.8e-06 | 196418
  27 |   5.2e-06 | 317811
  28 |     5e-06 | 514229
  29 |     6e-06 | 832040
  30 |     6e-06 | 1346269
  31 |   5.2e-06 | 2178309
  32 |     6e-06 | 3524578
  33 |   6.2e-06 | 5702887
  34 |   5.7e-06 | 9227465
  35 |   6.9e-06 | 14930352


<img src="img/19-20_Fc.png" style="width: 50em" />  

You should see (and explicitly check) that the algorithm now runs in linear time.  
It simply descends one branch of the the tree to a single set of leaf nodes, $F(1)$ and $F(0)$, keeping other branches unexplored.  
Once it hits the base cases and solves for $F(2)$, it updates the cache and ascends by one level.  
Now the next unexplored branch can see the new cache and make a direct calculation as well.  
This continues all the way up, leapfrogging between cache update and function evaluation, so the ascent is also single-pass. 

**Note:** Now that the speed is ok, you can reach higher values of $n$ (say up to $100$).  
In that case, you should worry about stability and typing errors because the sums get quite large.  
So replacing the cache array (a Python `list`) with a `numpy` array (`dtype=int`) is fine, up to around $n=90$; check for yourself if you like.    
Beyond that, the `numpy` array has overflow issues because its integers are size limited (and $1.6^{90} \approx 2^{64}$).  
Default Python integers are actually of type-`bignum` and have no intrinsic size limit, which is why I avoided `numpy` here.  

### Aside: Python Caching

Above I translated Skiena's caching work (originally in C) to Python, but there is also a [Python-specific option](https://docs.python.org/3/library/functools.html) in the form of `functools.cache`.   
Here I applied it as a decorator to the naive recursion code, and it will create an internal cache to store the values of previous function calls.  

This has a few benefits: easy to apply from naive version, you don't have to explicitly set cache size, it's threadsafe, etc.  
Evidently it can also be done at no noticeable loss to performance.  
The main downside is that accessing the cache itself is not really supported, so you might have issues debugging with it.  
One other downside is that it's Python specific, so it does not translate directly if you need to make the equivalent in other languages like C or Java.  

Note as well that becasue the cache is internal, $F(n)$ stores it for the session (not just the function call) until you clear the cache.  
This is a blessing in that calculating say $F(100)$ is only slow when the cache is empty (i.e. first few times you call it), and immediate for all subsequent calls.  
It is a curse in that you actually have to manage the memory usage yourself (e.g. the garbage collector will not automatically remove it). 

In [4]:
# same as naive code, but with a cache decorator

@cache
def F_rec2(n):
    if (n == 0)|(n == 1):
        return 1
    else:
        return F_rec2(n-1) + F_rec2(n-2) 

F_rec2.cache_clear()
print('   n | time[sec] | F(n)')
for i in range(0, 36): 
    start = time.time()
    temp = F_rec2(i)
    end = time.time()
    print(str(i).rjust(4), '|', str(np.round(end - start, 7)).rjust(9), '|', str(temp))

   n | time[sec] | F(n)
   0 |   1.2e-06 | 1
   1 |     1e-06 | 1
   2 |     1e-06 | 2
   3 |     1e-06 | 3
   4 |       0.0 | 5
   5 |       0.0 | 8
   6 |       0.0 | 13
   7 |       0.0 | 21
   8 |       0.0 | 34
   9 |       0.0 | 55
  10 |       0.0 | 89
  11 |       0.0 | 144
  12 |       0.0 | 233
  13 |       0.0 | 377
  14 |     1e-06 | 610
  15 |       0.0 | 987
  16 |     1e-06 | 1597
  17 |       0.0 | 2584
  18 |     1e-06 | 4181
  19 |       0.0 | 6765
  20 |       0.0 | 10946
  21 |       0.0 | 17711
  22 |       0.0 | 28657
  23 |       0.0 | 46368
  24 |     1e-06 | 75025
  25 |       0.0 | 121393
  26 |       0.0 | 196418
  27 |       0.0 | 317811
  28 |       0.0 | 514229
  29 |   1.2e-06 | 832040
  30 |       0.0 | 1346269
  31 |     1e-06 | 2178309
  32 |       0.0 | 3524578
  33 |       0.0 | 5702887
  34 |       0.0 | 9227465
  35 |       0.0 | 14930352


In [5]:
F_rec2.cache_info()

CacheInfo(hits=68, misses=36, maxsize=None, currsize=36)

In [6]:
# Single-shot times for reference

print('Manual cache')
%time a = F_cache_driver(100)

print('\nCache decorator')
F_rec2.cache_clear()
%time a = F_rec2(100)


Manual cache
CPU times: user 26 µs, sys: 9 µs, total: 35 µs
Wall time: 36.2 µs

Cache decorator
CPU times: user 26 µs, sys: 1 µs, total: 27 µs
Wall time: 27.2 µs


## Bottom-Up Dynamic Programming

We can tighten up the algorithm by noting: 
1. We don’t need to store all prior values.  
   Only the previous 2 values in the sequence are needed to calculate the next, and the rest can be safely forgotten.  
1. We know the ultimate order we need sub-answers, so we can just skip the descent portion and skip straight to ascent.

In our previous version, we needed $O(n)$ space to make an $n$-length cache array that holds all the values from $F(0)$ to $F(n)$.  
But the Fibonacci numbers are generated using only values up to 2 positions away.  
So we can safely drop to a size-2 cache array by just storing the last 2 calculated values ($O(1)$ in space). 

Our previous version also started trying to calculate $F(N)$ and worked downward to leaf-level before going back up (top-down approach).  
It does this to automatically determine an efficient order of solving sub-problems and building the call stack.  
But we don't need to have the computer do this; we can just tell it which way to go, especially if we know the most efficient order beforehand.  
So we can replace the recursive calls with a `for`-loop starting from the base cases.  
This keeps $O(n)$ time complexity, but has a better scaling coefficient because we skipped the descent phase.

Below I have some demo code for the ultimate version of the Fibonacci code that uses both of these improvements.  
Again, you can explicitly check the scaling is linear.  
Furthermore the single-shot calculation of $F(100)$ is a factor of $\approx 3$ faster than the top-down approach.  

In [7]:
def F_ult(N):
    cache = [1, 1]
    for i in range(2, N+1):
        out = sum(cache)
        cache = [cache[1], out]
    return cache[1]

print('   n | time[sec] | F(n)')
for i in range(0, 36): 
    start = time.time()
    temp = F_ult(i)
    end = time.time()
    print(str(i).rjust(4), '|', str(np.round(end - start, 7)).rjust(9), '|', str(temp))

   n | time[sec] | F(n)
   0 |     1e-06 | 1
   1 |       0.0 | 1
   2 |   2.1e-06 | 2
   3 |     1e-06 | 3
   4 |   2.1e-06 | 5
   5 |     1e-06 | 8
   6 |     1e-06 | 13
   7 |   1.2e-06 | 21
   8 |   1.2e-06 | 34
   9 |     1e-06 | 55
  10 |   1.2e-06 | 89
  11 |     7e-07 | 144
  12 |     1e-06 | 233
  13 |   1.2e-06 | 377
  14 |   1.9e-06 | 610
  15 |     1e-06 | 987
  16 |   1.2e-06 | 1597
  17 |     1e-06 | 2584
  18 |   1.9e-06 | 4181
  19 |   2.1e-06 | 6765
  20 |   2.1e-06 | 10946
  21 |   1.9e-06 | 17711
  22 |   3.1e-06 | 28657
  23 |   2.1e-06 | 46368
  24 |   1.9e-06 | 75025
  25 |   3.1e-06 | 121393
  26 |   2.6e-06 | 196418
  27 |   3.1e-06 | 317811
  28 |   3.1e-06 | 514229
  29 |   2.9e-06 | 832040
  30 |   4.1e-06 | 1346269
  31 |   4.1e-06 | 2178309
  32 |   2.9e-06 | 3524578
  33 |   4.3e-06 | 5702887
  34 |   2.9e-06 | 9227465
  35 |   3.1e-06 | 14930352


In [8]:
%time F_ult(100)

CPU times: user 12 µs, sys: 0 ns, total: 12 µs
Wall time: 14.1 µs


573147844013817084101

# Example: Binomial Coefficients

Recall (from combinatorial search) that some problems grow as binomial coefficients $\binom{n}{k}$ which we want to calculate.  
These have a theoretically convenient expression in terms of factorials $$\binom{n}{k} = \dfrac{n!}{k!(n-k)!}$$
but this is computationally inconvenient because you would have to calculate 3 factorials and take their ratio.  
Even if you have an efficient implementation of factorial calculation, the numerator and denominator can grow quickly.  
Aside from size limits of integer types, calculating large numbers and taking their ratio always raises issues of stability.  

Better would be to have a recursive relation on the binomial coefficients themselves.   
We do have one in the form of Pascal's rule $$\binom{n}{k} = \binom{n-1}{k-1} + \binom{n-1}{k}$$
This relationship has a convenient representation in the form of [Pascal's triangle](https://en.wikipedia.org/wiki/Pascal%27s_triangle).  
For now I will skip over how to get this relation, just claim it exists, and move on to the code. 

## Naive Recursion

Again, we start with the naive approach which we know should work for at least small cases.  
Below I have some demo code.  

As we expected, it works, but it slows down as $n$ increases, and as $k \rightarrow n/2$.  
We see from the triangle representation that the algorithm only requires $\approx nk$ items, so we could potentially go as fast as $O(nk)$.  
But without any shared calculations, the algorithm depends solely on the branching factor and tree depth, i.e. $O(2^n)$.  

In [9]:
def binom_n(n, k):
    if k > n:
        return 0
    if (k == 0)|(k == n):
        return 1
        
    return binom_n(n - 1, k - 1) + binom_n(n - 1, k)

# check we can make a Pascal triangle

n = 19
spacer = 5

for i in range(n+1):
    out = ''
    for j in range(i+1):
        out += (str(binom_n(i, j)).rjust(spacer)+' ') 
    print(out)


    1 
    1     1 
    1     2     1 
    1     3     3     1 
    1     4     6     4     1 
    1     5    10    10     5     1 
    1     6    15    20    15     6     1 
    1     7    21    35    35    21     7     1 
    1     8    28    56    70    56    28     8     1 
    1     9    36    84   126   126    84    36     9     1 
    1    10    45   120   210   252   210   120    45    10     1 
    1    11    55   165   330   462   462   330   165    55    11     1 
    1    12    66   220   495   792   924   792   495   220    66    12     1 
    1    13    78   286   715  1287  1716  1716  1287   715   286    78    13     1 
    1    14    91   364  1001  2002  3003  3432  3003  2002  1001   364    91    14     1 
    1    15   105   455  1365  3003  5005  6435  6435  5005  3003  1365   455   105    15     1 
    1    16   120   560  1820  4368  8008 11440 12870 11440  8008  4368  1820   560   120    16     1 
    1    17   136   680  2380  6188 12376 19448 24310 24310 19448

In [10]:
# Check execution time 

n = 26 # default 26, go higher if you can wait more

print(' k | time[sec] | C(n, k)')
for j in range(n+1):
    start = time.time()
    temp = binom_n(n, j)
    end = time.time()
    print(str(j).rjust(2), '|', str(np.round(end - start, 7)).rjust(9), '|', str(temp))

 k | time[sec] | C(n, k)
 0 |   1.2e-06 | 1
 1 |   6.9e-06 | 26
 2 |   6.1e-05 | 325
 3 | 0.0004969 | 2600
 4 | 0.0026999 | 14950
 5 | 0.0121498 | 65780
 6 | 0.0447628 | 230230
 7 | 0.1168981 | 657800
 8 |  0.272402 | 1562275
 9 | 0.5480411 | 3124550
10 |  0.939687 | 5311735
11 |  1.370934 | 7726160
12 |  1.712306 | 9657700
13 |  1.856797 | 10400600
14 | 1.7169552 | 9657700
15 | 1.3720059 | 7726160
16 | 0.9413192 | 5311735
17 | 0.5545049 | 3124550
18 | 0.2764149 | 1562275
19 | 0.1169748 | 657800
20 | 0.0401759 | 230230
21 | 0.0114331 | 65780
22 |  0.002605 | 14950
23 |  0.000457 | 2600
24 |  5.72e-05 | 325
25 |   4.8e-06 | 26
26 |       0.0 | 1


## Tabulation  

We can speed this up with a (bottom-up) dynamic programming approach.  
In this case, the most convenient ordering follows the pyramid, increasing in $k$, and incrementing $n$ as soon as $k=n$.  
Because of the relation, we know that $\binom{n}{k}$ only depends on coefficients with smaller values of $n$ and $k$ so we will need $O(nk)$ space at most.  

The simplest storage space we could construct with this is an $n \times k$ table.  
We would fill it up with the entries of a triangle, so it is slightly wasteful, but it will at least work.  
To actually fill each entry, we would just take the sum of the element directly above and north-west of the current position.  
Each entry other than the base cases will be calculated exactly once by arithmetic, which is $O(1)$, so the entire algorithm should run in $O(nk)$.  

Below I have written some demo code to illustrate.  
As you will see, it only starts *barely* slowing down at $n \approx 100$ in the worst case.  
In fact it's still 4 orders of magnitude faster than the naive version at $n=26$.  
**Moral of the story:** Respect exponential scaling, and avoid it like the plague where possible. 

Again, this isn't even the most optimal version of this code.  
Some things you could do:
- Instead of storing a whole table, you could get away with just storing the last row calculated. 
- Instead of storing a length-$k$ row, the length of the row can shrink as you get closer to either the tip or $n$ itself (because of the triangle shape).  
- Instead of going top-down, you could find a different recursion relation to go from the side of the triangle.  
In that case, you would only need to calculate $\text{min}(k, n-k)$ entries.  

That said, this should be good enough for most mundane tasks (though if you're going for a high score or something, by all means).  

In [11]:
def binom_tab(n, k):
    table = [[0]*(k+1) for _ in range(n + 1)]
    for i in range(n + 1):
        for j in range(k + 1): 
            if (j == 0)|(j == i):
                table[i][j] = 1
            else:
                table[i][j] = table[i-1][j-1] + table[i-1][j]

    return table[n][k]

print(' n | time[sec] | C(n, n//2)')
for n in range(0, 100):
    start = time.time()
    temp = binom_tab(n, n//2)
    end = time.time()
    print(str(n).rjust(2), '|', str(np.round(end - start, 7)).rjust(9), '|', str(temp))

 n | time[sec] | C(n, n//2)
 0 |   1.9e-06 | 1
 1 |   1.7e-06 | 1
 2 |   2.1e-06 | 2
 3 |   1.9e-06 | 3
 4 |   2.9e-06 | 6
 5 |   3.1e-06 | 10
 6 |     5e-06 | 20
 7 |     6e-06 | 35
 8 |   6.2e-06 | 70
 9 |   7.2e-06 | 126
10 |   9.1e-06 | 252
11 |     1e-05 | 462
12 |  1.26e-05 | 924
13 |  1.29e-05 | 1716
14 |  1.53e-05 | 3432
15 |  1.57e-05 | 6435
16 |  1.91e-05 | 12870
17 |     2e-05 | 24310
18 |  2.26e-05 | 48620
19 |  2.41e-05 | 92378
20 |  2.79e-05 | 184756
21 |  2.81e-05 | 352716
22 |  3.29e-05 | 705432
23 |  3.41e-05 | 1352078
24 |  3.89e-05 | 2704156
25 |  3.98e-05 | 5200300
26 |  4.41e-05 | 10400600
27 |  4.72e-05 | 20058300
28 |  5.08e-05 | 40116600
29 |  5.17e-05 | 77558760
30 |  5.72e-05 | 155117520
31 |  5.89e-05 | 300540195
32 |  6.51e-05 | 601080390
33 |   6.7e-05 | 1166803110
34 |   7.2e-05 | 2333606220
35 |  7.39e-05 | 4537567650
36 |  8.23e-05 | 9075135300
37 |   8.3e-05 | 17672631900
38 |  9.11e-05 | 35345263800
39 |   9.2e-05 | 68923264410
40 |  9.99e-05 | 1378465

## Formulating Recurrences 

Remember: dynamic programming is applicable when the problem has overlapping sub-problems.  
That overlap **is** a recurrence relation, and knowing how to express it explicitly and mathematically is how you start hunting out inefficiencies. 

With that in mind, you will at some point probably need to make/derive a recurrence relation on your own.  
For the Fibonacci numbers, the sequence is defined by the relation, so that much is taken care of.  
But for calculating binomial coefficients, we used Pascal's rule - what if we didn't know it existed?  
What if you're working on a problem no one's done before (or at least one highly specific to your job) and *no one* has found a relation? 

At some point, finding and exploiting recurrence relations for coding a dynamic programming strategy can become instinctual.  
It just requires a lot of practice (which I personally do not posses any more).  
Until then, I have 2 general strategies for finding them:
- Brute force \& ignorance 
- Induction 

### Brute Force \& Ignorance
This approach is pretty simple:  
1. Manually solve (smaller versions of) the problem.  
1. List out all the intermediate calculations as a table or tree.  
1. Look at the table/tree until you notice a pattern or spot to optimize.
1. Test your pattern/optimization, and rationalize it in hindsight.  

This is definitely the slower method, and it may not even work depending on your current skill level.  
But I do genuinely recommend you do this every now and then to develop your instincts in the long term, especially for very unfamiliar problems.  
If nothing else, it at least forces you to think about the problem for longer, which is where a lot of breakthroughs happen.  

### Induction
The other, more structured approach, is similar to the proof strategy:  
1. Find the base cases. 
1. For a sub-problem of index or size $i$, relate it to the case $i\pm1$.  

As an example, consider calculating the binomial coefficients.  
Binomial coefficients $\binom{n}{k}$ are the number of to find ways to choose (unordered) sets of $k$ items from $n$ options.  
So we can identify the 2 base cases as
- $(k=0 ,\quad \forall n)$ : only one way to pick nothing.
- $(k=n ,\quad \forall n)$ : only one way to pick everything.

Then the inductive step: we want to relate a case of some size to a marginally smaller or larger one.  
Here we ultimately want to calculate $\binom{n}{k}$, and we know the base cases are $k=0$ and $k=n$.  
It stands to reason we should try to get a relation that brings us closer to the base cases, i.e. relate $\binom{n}{k}$ to smaller values of $n$ and $k$.  
Now we have a choice of which variable to decrease in order to make the relation.  

For Pascal's rule, we try relating the cases $n \rightarrow n-1$, leaving $k$ alone for now.  
This has a [combinatorial interpretation](https://en.wikipedia.org/wiki/Pascal%27s_rule#Combinatorial_proof).  
If one were to choose $k$ items from $n$ options, then (strictly) either all $k$ items are from a subset of size $n-1$ or only $k-1$ are from that subset.  
The number of ways for the former to occur is $\binom{n-1}{k}$, while for the latter it is $\binom{n-1}{k-1}$.  
So adding them up will make $\binom{n}{k}$ and Pascal's rule is recovered.  

<img src="img/19-20_pr.png" style="width: 30em" />  

Alternatively, we could leave $n$ alone and change $k$.  
This is harder to draw because values increase/decrease depending on the position in triangle.   
But you can always fall back on the factorial definition of the coefficients, at which point wrangling into the desired form should give the relation. 
\begin{align}
\binom{n}{k} &= \dfrac{n!}{k!(n-k)!}\\ 
&= \dfrac{n!}{(k-1)!(n-k)!}\times \left[\dfrac{1}{k}\right]\\ 
&= \dfrac{n!}{(k-1)!(n-k+1)!}\times \left[\dfrac{n-k+1}{k}\right]\\ 
&= \dfrac{n!}{(k-1)!(n-(k-1))!}\times \left[\dfrac{n-k+1}{k}\right]\\ 
&= \binom{n}{k-1}\times \left[\dfrac{n-k+1}{k}\right]
\end{align}

So this would give a different yet equally valid relationship.    
If you went in the other direction to relate to $k+1$, you should get a similar propotionality.  

# Approximate String Matching 

Recall, from the section on hashing, that a common problem is matching strings and substrings. 
In that case, we needed to find an exact match in order for the method to work.  
In practice, there may be typos (when you fat-finger the keyboard) or misspellings (intentional or otherwise).  
Or, some words may only be vaguely similar but convey the same meaning (e.g. “y’all” vs “yinz”).  

## Edit Distance

To handle inexact matches, we need a metric (read: disance) of dissimilarity between strings.  
First we need to lay out the operations available to edit a string into greater/lesser similarity with another.  
The $3$ options we will consider are:  
- **Substitution** – Change a single character to a different character.  
Ex. "hour" to "pour"
- **Insertion** – Insert a single character at any position.  
Ex. "our" to "pour"
- **Deletion** – Delete a single character at any position.  
Ex. "hour" to "our"

If each operation is assigned a penalty/distance of $1$, their sum makes the **(Levenshtein) edit distance**.  
For example, "stop" and "start" has an edit distance of $3$ between them
1. Delete the last "t" from "start" $\rightarrow$ "star".
1. Substitute "p" for "r" in "star" $\rightarrow$ "stap".
1. Substitute "o" for "a" in "stap" $\rightarrow$ "stop".

Clearly we can insert and delete any character over and over to artificially inflate the distance.  
So when we refer to the edit distance, we generally mean the smallest possible value it can take.   
This is not the only way to assign distances, and you may explore some variations later, but it is the simplest.

## Naive Recursion

To actually calculate this edit distance, consider the ending character of both strings and check if it they match or not.    
If they are matched, we don't need to edit it and can move on to the other characters (contribution of $0$ to the edit distance). 
When they are not matched, the appropriate edit will depend on all the characters before it, the "prefix" of this character. 
If we knew the best prefix, we can pick option that minimizes the distance for this character.  
As with Dijkstra’s algorithm, this suggests a recursive approach.  
1. For matched characters, they do not contribute to edit distance and can be ignored.  
1. For unmatched characters, test all 3 edit options to match the last character.  
1. To get the correct option, recur on the prefix to get its edit distance, then take minimum.   

Below I have some demo code for this, comparing 2 totally dissimilar strings of equal length.  
You'll immediately notice that it is hopelessly slow beyond strings of length 10.  
Again, the naive version doesn't share calculations so go as an exponential of branching factor ($=3$).  
We already had issues with branching factors of $1.6$ and  $2$, so it's no surprise this one is as slow as it is.  

In [12]:
def lev_rec(s ,t): 
    # base cases: if either string is empty, the edit distance is purely from insertions
    if len(s) == 0:
        return len(t)
    if len(t) == 0:
        return len(s)
        
    if s[-1] == t[-1]:
        return lev_rec(s[:-1], t[:-1])
    else:
        return 1+min([
                    lev_rec(s[:-1], t), 
                    lev_rec(s, t[:-1]),
                    lev_rec(s[:-1], t[:-1])
                    ])
        
print(' i | time[sec] | lev(s, t)')
for i in range(0, 11):
    s = 'a'*i
    t = 'b'*i
    
    start = time.time()
    temp = lev_rec(s, t)
    end = time.time()
    print(str(i).rjust(2), '|', str(np.round(end - start, 7)).rjust(9), '|', str(temp))

 i | time[sec] | lev(s, t)
 0 |       0.0 | 0
 1 |   1.7e-06 | 1
 2 |     5e-06 | 2
 3 |     2e-05 | 3
 4 | 0.0001042 | 4
 5 | 0.0004752 | 5
 6 |  0.002511 | 6
 7 | 0.0140162 | 7
 8 |  0.069299 | 8
 9 | 0.3829899 | 9
10 |  2.099762 | 10


## Tabulation: Prefix Matrix

Dynamic programming is a good approach here because: 
1. As soon as a correct edit is made, we never want to revisit it.  
1. Many proposed edits are shared in the search tree.  
In fact, substitution (a 1-distance edit) is degenerate with deletion followed by insertion of a character (a 2-distance edit).  

First we should pick a convenient representation for the storage (cache or table). 
The most natural approach is tabulation/bottom-up approach because
1. We know the base cases (empty prefix)
1. It's easier to think about the effect of changing a character on smaller prefixes.  
These suggest storing results in a (prefix) distance matrix. 

In the slides, we construct one manually just to drill in the idea, but it can also be done theoretically by translating the recursive updates.   
- The matrix is an $(n+1) \times (m+1)$ matrix, whose rows and columns correspond to prefixes of increasing length.   
- The first row and column contain the base cases (the empty prefix), so they are filled linearly increasing.  
- Per unfilled position, check the ending characters of the prefixes.
    - If matched, the distance is exactly that of the prefixes one size smaller, i.e. the element directly north-west of it.
    - If mismatched, the distance is $1 + \text{min}(\text{<3 options>})$.  
      The $+1$ is for the corresponding edit one needs to do.  
      The 3 options are the 3 ways to approach the current position from smaller prefixes: direct north, direct west, or direct north-west.
- The final edit distance is the bottom right-most element.

Below I have some demo code to implement this approach.  
We use a `for`-loop to fill all entries of the matrix, top-to-bottom and left-to-right.   
Per matrix position `(i, j)`, run an `IF`-statement to check if the end characters match.   
If matched, copy the value in position `(i-1, j-1)`.  
Otherwise, take $1+$ minimum of values in positions `[(i-1, j-1), (i, j-1), (i-1, j)]`.  
Since every item is obtained by arithmetic (which is $O(1)$) once, and there are $mn$ of them, this should run in $O(mn)$ time.  
 
Again, there are some optimizations or alternatives you could do.  
Some options for you to explore:
- Could use a top-down approach, as in the Fibonacci problem
- Can optimize space usage by only storing the previous row/column, since only the adjacent row/column is used in calculating entries.
- Most of the table entries have absolutely nothing to do with the ultimate path, so you can try to ignore those.  


In [13]:
def lev_tab(s, t):
    m = len(s)
    n = len(t)
    if m == 0:
        return n
    if n == 0:
        return m

    # prefix matrix
    pm = [[0]*(n + 1) for _ in range(m + 1)]

    # Fill in base cases
    for i in range(m + 1):
        pm[i][0] = i
    for j in range(n + 1):
        pm[0][j] = j

    # Fill in order, skipping base cases
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s[i - 1] == t[j - 1]:
                pm[i][j] = pm[i - 1][j - 1]
            else:
                pm[i][j] = 1 + min([
                                    pm[i][j - 1], 
                                    pm[i - 1][j], 
                                    pm[i - 1][j - 1]
                                    ])

    # If you want to check against lecture, return full matrix
    return pm[m][n]

print(' i | time[sec] | lev(s, t)')
for i in range(0, 50):
    s = 'a'*i
    t = 'b'*i
    
    start = time.time()
    temp = lev_tab(s, t)
    end = time.time()
    print(str(i).rjust(2), '|', str(np.round(end - start, 7)).rjust(9), '|', str(temp))

 i | time[sec] | lev(s, t)
 0 |       0.0 | 0
 1 |   3.8e-06 | 1
 2 |   3.8e-06 | 2
 3 |   3.8e-06 | 3
 4 |   6.9e-06 | 4
 5 |   9.1e-06 | 5
 6 |  1.31e-05 | 6
 7 |   1.6e-05 | 7
 8 |  1.98e-05 | 8
 9 |  2.41e-05 | 9
10 |  2.98e-05 | 10
11 |   3.5e-05 | 11
12 |   4.2e-05 | 12
13 |  4.82e-05 | 13
14 |  5.58e-05 | 14
15 |  6.29e-05 | 15
16 |   7.2e-05 | 16
17 |  8.08e-05 | 17
18 |  8.92e-05 | 18
19 |  9.92e-05 | 19
20 | 0.0001099 | 20
21 | 0.0001199 | 21
22 | 0.0001321 | 22
23 |  0.000144 | 23
24 | 0.0001559 | 24
25 | 0.0001678 | 25
26 | 0.0001819 | 26
27 |  0.000196 | 27
28 |   0.00021 | 28
29 | 0.0002251 | 29
30 | 0.0002472 | 30
31 |  0.000257 | 31
32 |  0.000272 | 32
33 | 0.0002902 | 33
34 | 0.0003068 | 34
35 | 0.0003252 | 35
36 |  0.000344 | 36
37 |  0.000361 | 37
38 |  0.000381 | 38
39 | 0.0004022 | 39
40 | 0.0005093 | 40
41 |  0.000452 | 41
42 | 0.0004661 | 42
43 | 0.0004861 | 43
44 | 0.0005097 | 44
45 | 0.0005329 | 45
46 |  0.000576 | 46
47 | 0.0006051 | 47
48 |  0.000653 | 48
49 

# Ending Notes

As mentioned at the start, dynamic programming is more about methodology than standard solutions.   
That's not to say there *aren't* standard problems, there definitely are.  
[Here](https://www.geeksforgeeks.org/top-20-dynamic-programming-interview-questions/) is a list of 20 that might come up in a job interview.  
Some notable ones:
- Knapsack Problem
- Rod Cutting Problem
- Coin Change Problem	
- Partition Problem

What I am saying is that it is largely impractical to just memorize solutions to all 20 of them, which is probably only scratching the surface anyway.  
It is certainly impractical for me to lecture on all of them (we don't have that kind of time).  
Furthermore, most technical interviews probably aren't going to ask these problems exactly - they'll tweak it in some ways that need adaptation.  

But if you can do at least some of these problems on your own, from scratch, you are much more likely to solve *any* of them.  
The ultimate goal is to build up a your base ability so that, even if you don't train for it exactly, you can still solve a generic problem.  
When you get to the stretch before a technical interview, *then* you can go wild and memorize as much as you want to squeeze out the last bits of performance. 

With that said, this will be the last planned lecture notebook; there should still be some slides for the last topic.  
The next sessions will mostly be exercises.  
I will give the outline of some of the dynamic programming problems above.  
Your goal will be to solve it with as little intervention as possible.  
And just to clarify, these in-class exercises will **not** be graded, these are purely for your learning.  
Personally, I would recommend making a notebook or report per problem.  

I have also reserved some problems that don't really fit in with the practice problems.  
These require a bit more independent work, and additional time to understand.  
If you wish to attempt these, notify me and I will assign one to you.  
The output is a report, and the reward is an oral exam.  